# Импорт необходимых библиотек

In [3]:
pip install boto3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 1.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 56.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.2/85.2 kB 17.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from dotenv import load_dotenv
import boto3
from pyspark.sql import SparkSession
import requests
import logging

# Подключение к Minio через библиотеку boto3 

In [3]:
load_dotenv()

# Получаем переменные
endpoint = "minio:9000" # для airflow нужно будет переписать чтобы значения брались из Variables
access_key = "minio"
secret_key = "minio123"

# Подключаемся к MinIO
s3 = boto3.client(
    's3',
    endpoint_url=f"http://{endpoint}",
    aws_access_key_id=access_key,
    aws_secret_access_key=secret_key

)

# Проверяем список бакетов
try:
    buckets = s3.list_buckets()
    print("Подключение к MinIO через boto3 успешно. Список бакетов:")
    for b in buckets['Buckets']:
        print(f"  - {b['Name']}")
except Exception as e:
    print("Ошибка подключения к MinIO:", e)

Подключение к MinIO через boto3 успешно. Список бакетов:
  - new


# Подключение к Minio через библиотеку Minio 

In [ ]:
client = Minio(
    endpoint=os.getenv("MINIO_ENDPOINT"), # для airflow нужно будет переписать чтобы значения брались из Variables
    access_key=os.getenv("MINIO_ACCESS_KEY"),
    secret_key=os.getenv("MINIO_SECRET_KEY"),
    secure=False
)

# Проверяем список бакетов
try:
    buckets = client.list_buckets()
    print("Подключение к MinIO через MinIO успешно. Список бакетов:")
    for b in buckets:
        print(f' - {b.name}')
except Exception as e:
    print("Ошибка подключения к MinIO:", e)

NameError: name 'Minio' is not defined

# Проверка SparkSession 

In [43]:
spark = SparkSession.builder \
    .appName("MinIO Test") \
    .getOrCreate()

df = spark.range(10)
df.show()

+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
|  7|
|  8|
|  9|
+---+



# Запуск SparkSession с различными параметрами 

**Имя приложения, которое будет отражено в Spark UI и логах**

appName("SparkProject")

**Локальный режим (может быть yarn, mesos, kubernetes)**

master("local[*]") 

**Количество партиций, используемых при операциях shuffle (например, groupBy, join). По умолчанию = 200.
Для небольших наборов данных можно уменьшить, чтобы не было лишних партиций.**

config("spark.sql.shuffle.partitions", "200") 

**Объем памяти, выделяемый каждому executor'у. Чем больше данных — тем больше памяти потребуется.**

config("spark.executor.memory", "4g")

**Объем памяти для драйвера — основной управляющий процесс, запускающий задачи и отслеживающий их выполнение.**

config("spark.driver.memory", "2g") 

**Количество ядер CPU, которые каждый executor может использовать. Больше ядер — больше параллелизма, но стоит учитывать общую нагрузку.**

config("spark.executor.cores", "4")

**Включает динамическое выделение executors в зависимости от загрузки. Позволяет Spark автоматически масштабироваться: добавлять и убирать executors в зависимости от нужд приложения. Требует настройки shuffle service.**

config("spark.dynamicAllocation.enabled", "true")

**Указывает директорию для checkpoint'ов (резервного сохранения состояния, особенно в streaming-приложениях).**

config("spark.checkpoint.dir", "/path/to/checkpoint/dir")

**Путь к директории, где Spark SQL будет хранить управляемые таблицы (при использовании saveAsTable, например).**

config("spark.sql.catalogImplementation", "hive")


**Указывает, что Spark должен использовать Hive Catalog (а не встроенный In-Memory метастор). Требует наличие Hive или его имитации (например, Derby DB).**

.enableHiveSupport()


In [ ]:
# spark = SparkSession.builder \
#     .appName("SparkProject") \
#     .master("local[*]") \
#     .config("spark.sql.shuffle.partitions", "200") \
#     .config("spark.executor.memory", "4g") \
#     .config("spark.driver.memory", "2g") \
#     .config("spark.executor.cores", "4") \
#     .config("spark.dynamicAllocation.enabled", "true") \
#     .config("spark.checkpoint.dir", "/path/to/checkpoint/dir") \
#     .config("spark.sql.warehouse.dir", "/path/to/warehouse/dir") \
#     .config("spark.sql.catalogImplementation", "hive") \
#     .getOrCreate()

# Получаем данные по API

In [6]:
def get_api_data(env_name):
    load_dotenv()
    # Получаем ссылку из файла .env
    api_url = os.getenv(env_name) # для airflow нужно будет переписать чтобы значения брались из Variables
    # Проверка наличия ссылки
    if not api_url:
        logging.error(f'Указанный URL не задан в файле .env') # для airflow нужно будет переписать чтобы значения брались из Variables
        return None
    try:
        response = requests.get(api_url)
        if response.status_code == 200:
            data = response.json()
            json_str = json.dumps(data, ensure_ascii=False, indent=2)
            logging.info('Данные успешно загружены')
            return json_str
        else:
            logging.error(f'Error: {response.status_code} - Данные не загружены')
            return None
    except:
        logging.error(f'Error{e} - Данные не загружены')
        raise



# Сохраняем JSON в MinIO

In [ ]:
def save_json_to_minio(env_name):
    data = get_api_data(env_name)

# Написать функцию, чтобы записать в Minio JSON
# Проверить в какой бакет пишем перезаписью
# Через Spark читать файл


In [8]:
!curl minio:9001

<!doctype html><html lang="en"><head><meta charset="utf-8"/><base href="/"/><meta content="width=device-width,initial-scale=1" name="viewport"/><meta content="#081C42" media="(prefers-color-scheme: light)" name="theme-color"/><meta content="#081C42" media="(prefers-color-scheme: dark)" name="theme-color"/><meta content="MinIO Console" name="description"/><meta name="minio-license" content="agpl"/><link href="./styles/root-styles.css" rel="stylesheet"/><link href="./apple-icon-180x180.png" rel="apple-touch-icon" sizes="180x180"/><link href="./favicon-32x32.png" rel="icon" sizes="32x32" type="image/png"/><link href="./favicon-96x96.png" rel="icon" sizes="96x96" type="image/png"/><link href="./favicon-16x16.png" rel="icon" sizes="16x16" type="image/png"/><link href="./manifest.json" rel="manifest"/><link color="#3a4e54" href="./safari-pinned-tab.svg" rel="mask-icon"/><title>MinIO Console</title><script defer="defer" src="./static/js/main.8ed8a7ba.js"></script><link href="./static/css/main

In [ ]:
spark = SparkSession.builder \
    .appName("SparkProject") \
    .master("local[*]") \
    .config("spark.sql.shuffle.partitions", "200") \
    .config("spark.driver.memory", "2g") \
    .config("spark.dynamicAllocation.enabled", "true") \
    .getOrCreate()

In [15]:
import os

In [16]:
os.getcwd()

'/home/jovyan/notebooks'

In [6]:
from io import BytesIO
from pathlib import Path
from typing import Optional, Union

import boto3
from botocore.client import Config
from botocore.exceptions import ClientError
from botocore.response import StreamingBody


class S3BucketService:
    def __init__(
        self,
        bucket_name: str,
        endpoint: str,
        access_key: str,
        secret_key: str,
    ) -> None:
        self.bucket_name = bucket_name
        self.endpoint = endpoint
        self.access_key = access_key
        self.secret_key = secret_key

    def create_s3_client(self) -> boto3.client:
        client = boto3.client(
            "s3",
            endpoint_url=self.endpoint,
            aws_access_key_id=self.access_key,
            aws_secret_access_key=self.secret_key,
            config=Config(signature_version="s3v4"),
        )
        return client

    def upload_file_object(
        self,
        prefix: str,
        source_file_name: str,
        content: Union[str, bytes],
    ) -> None:
        client = self.create_s3_client()
        destination_path = str(Path(prefix, source_file_name))

        if isinstance(content, bytes):
            buffer = BytesIO(content)
        else:
            buffer = BytesIO(content.encode("utf-8"))
        client.upload_fileobj(buffer, self.bucket_name, destination_path)

    def list_objects(self, prefix: str) -> list[str]:
        client = self.create_s3_client()

        response = client.list_objects_v2(Bucket=self.bucket_name, Prefix=prefix)
        storage_content: list[str] = []

        try:
            contents = response["Contents"]
        except KeyError:
            return storage_content

        for item in contents:
            storage_content.append(item["Key"])

        return storage_content

    def delete_file_object(self, prefix: str, source_file_name: str) -> None:
        client = self.create_s3_client()
        path_to_file = str(Path(prefix, source_file_name))
        client.delete_object(Bucket=self.bucket_name, Key=path_to_file)

In [7]:
def s3_bucket_service_factory() -> S3BucketService:
    return S3BucketService(
        'test',
        'http://localhost:9000',
        'minio',
        'minio',
    )

In [8]:

s3 = s3_bucket_service_factory()
#s3.upload_file_object("test", "test.txt", "test")

In [12]:
client = boto3.client(
    "s3",
    endpoint_url='http://minio:9000',
    aws_access_key_id='minio',
    aws_secret_access_key='minio123',
    config=Config(signature_version="s3v4"),
)

In [13]:
response = client.list_objects_v2(Bucket='new')

In [14]:
response

{'ResponseMetadata': {'RequestId': '18518918E08AD2E6',
  'HostId': 'dd9025bab4ad464b049177c95eb6ebf374d3b3fd1af9251148b658df7ac2e3e8',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'accept-ranges': 'bytes',
   'content-length': '672',
   'content-type': 'application/xml',
   'server': 'MinIO',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'vary': 'Origin, Accept-Encoding',
   'x-amz-id-2': 'dd9025bab4ad464b049177c95eb6ebf374d3b3fd1af9251148b658df7ac2e3e8',
   'x-amz-request-id': '18518918E08AD2E6',
   'x-content-type-options': 'nosniff',
   'x-ratelimit-limit': '16853',
   'x-ratelimit-remaining': '16853',
   'x-xss-protection': '1; mode=block',
   'date': 'Sat, 12 Jul 2025 15:01:09 GMT'},
  'RetryAttempts': 0},
 'IsTruncated': False,
 'Contents': [{'Key': 'cars.json',
   'LastModified': datetime.datetime(2025, 7, 12, 11, 15, 49, 810000, tzinfo=tzlocal()),
   'ETag': '"2c2c4b49bd2a3ed0faff8387664deaea"',
   'Size': 100492,
   'StorageClass': 'STANDARD'},
  {'Ke

In [19]:
from pyspark.sql import SparkSession, SQLContext
import pyspark.sql.functions as F
import logging
import os
from airflow.models import Variable
from dotenv import load_dotenv
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, DataType
import requests
import json
import boto3
import minio

In [ ]:
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

# Загрузка и проверка переменных окружения
load_dotenv()



def create_spark_session():
    try:
        spark = SparkSession.builder \
            .appName("MinIO Data Reader") \
            .config("spark.hadoop.fs.s3a.access.key", "minio") \
            .config("spark.hadoop.fs.s3a.secret.key", "minio123") \
            .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000") \
            .config("spark.hadoop.fs.s3a.path.style.access", "true") \
            .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false") \
            .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
            .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider") \
            .config("spark.jars", "/opt/jars/hadoop-aws-3.3.1.jar,/opt/jars/aws-java-sdk-bundle-1.11.901.jar") \
            .getOrCreate()


        logging.info('SparkSession успешно создана для работы с MinIO')
    except Exception as e:
        logging.error(f'SparkSession не создана по причине: {e}')
        raise
    return spark

In [ ]:
def download_to_clickhouse():
    path = "s3a://weather/weather_data.json"
    spark = create_spark_session()
    schema = StructType([
        StructField("latitude", DoubleType()),
        StructField("longitude", DoubleType()),
        StructField("timezone", StringType()),
        StructField("current_weather_units", StructType([
            StructField("time", StringType())
        ])),
        StructField("current_weather", StructType([
            StructField("time", TimestampType()),
            StructField("temperature", DoubleType()),
            StructField("windspeed", DoubleType())
    ]))
    ])
    raw_df = spark.read.json(path, schema=schema, multiLine=True)
    #raw_df.show(truncate=False)
    #raw_df.printSchema()
    flat_df = raw_df.select(
        "latitude", "longitude", "timezone",
        F.col("current_weather_units.time").alias("format_time"),
        F.col("current_weather.time").alias("time"),
        F.col("current_weather.temperature").alias("temperature"),
        F.col("current_weather.windspeed").alias("windspeed")

    )
    flat_df.show()
    return flat_df
download_to_clickhouse()

[2025-07-12T15:30:22.402+0000] {4070852412.py:26} INFO - SparkSession успешно создана для работы с MinIO


Py4JJavaError: An error occurred while calling o110.json.
: java.nio.file.AccessDeniedException: s3a://weather/weather_data.json: getFileStatus on s3a://weather/weather_data.json: com.amazonaws.services.s3.model.AmazonS3Exception: Forbidden (Service: Amazon S3; Status Code: 403; Error Code: 403 Forbidden; Request ID: 18518AB0E71077AF; S3 Extended Request ID: dd9025bab4ad464b049177c95eb6ebf374d3b3fd1af9251148b658df7ac2e3e8; Proxy: null), S3 Extended Request ID: dd9025bab4ad464b049177c95eb6ebf374d3b3fd1af9251148b658df7ac2e3e8:403 Forbidden
	at org.apache.hadoop.fs.s3a.S3AUtils.translateException(S3AUtils.java:249)
	at org.apache.hadoop.fs.s3a.S3AUtils.translateException(S3AUtils.java:170)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.s3GetFileStatus(S3AFileSystem.java:3286)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.innerGetFileStatus(S3AFileSystem.java:3185)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.getFileStatus(S3AFileSystem.java:3053)
	at org.apache.hadoop.fs.FileSystem.exists(FileSyst

In [25]:
spark.stop()

NameError: name 'spark' is not defined

In [27]:
path = "s3a://weather/weather_data.json"
spark = create_spark_session()

[2025-07-12T15:31:35.778+0000] {4070852412.py:26} INFO - SparkSession успешно создана для работы с MinIO


In [ ]:
schema = StructType([
    StructField("latitude", DoubleType()),
    StructField("longitude", DoubleType()),
    StructField("timezone", StringType()),
    StructField("current_weather_units", StructType([
        StructField("time", StringType())
    ])),
    StructField("current_weather", StructType([
        StructField("time", StringType()),
        StructField("temperature", DoubleType()),
        StructField("windspeed", DoubleType())
]))
])
raw_df = spark.read.json(path, schema=schema, multiLine=True)
#raw_df.show(truncate=False)
#raw_df.printSchema()
flat_df = raw_df.select(
    "latitude", "longitude", "timezone",
    F.col("current_weather_units.time").alias("format_time"),
    F.col("current_weather.time").alias("time"),
    F.col("current_weather.temperature").alias("temperature"),
    F.col("current_weather.windspeed").alias("windspeed")

)
flat_df.show()

Py4JJavaError: An error occurred while calling o164.json.
: java.nio.file.AccessDeniedException: s3a://weather/weather_data.json: getFileStatus on s3a://weather/weather_data.json: com.amazonaws.services.s3.model.AmazonS3Exception: Forbidden (Service: Amazon S3; Status Code: 403; Error Code: 403 Forbidden; Request ID: 18518B42E579DE4E; S3 Extended Request ID: dd9025bab4ad464b049177c95eb6ebf374d3b3fd1af9251148b658df7ac2e3e8; Proxy: null), S3 Extended Request ID: dd9025bab4ad464b049177c95eb6ebf374d3b3fd1af9251148b658df7ac2e3e8:403 Forbidden
	at org.apache.hadoop.fs.s3a.S3AUtils.translateException(S3AUtils.java:249)
	at org.apache.hadoop.fs.s3a.S3AUtils.translateException(S3AUtils.java:170)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.s3GetFileStatus(S3AFileSystem.java:3286)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.innerGetFileStatus(S3AFileSystem.java:3185)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.getFileStatus(S3AFileSystem.java:3053)
	at org.apache.hadoop.fs.FileSystem.exists(FileSyst

In [30]:
spark.

<bound method SparkSession.active of <class 'pyspark.sql.session.SparkSession'>>

In [35]:
path = "s3a://new/data.csv"
raw_df = spark.read.csv(path, multiLine=True)

Py4JJavaError: An error occurred while calling o182.csv.
: java.nio.file.AccessDeniedException: s3a://new/data.csv: getFileStatus on s3a://new/data.csv: com.amazonaws.services.s3.model.AmazonS3Exception: Forbidden (Service: Amazon S3; Status Code: 403; Error Code: 403 Forbidden; Request ID: 18518C2AE4F728F7; S3 Extended Request ID: dd9025bab4ad464b049177c95eb6ebf374d3b3fd1af9251148b658df7ac2e3e8; Proxy: null), S3 Extended Request ID: dd9025bab4ad464b049177c95eb6ebf374d3b3fd1af9251148b658df7ac2e3e8:403 Forbidden
	at org.apache.hadoop.fs.s3a.S3AUtils.translateException(S3AUtils.java:249)
	at org.apache.hadoop.fs.s3a.S3AUtils.translateException(S3AUtils.java:170)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.s3GetFileStatus(S3AFileSystem.java:3286)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.innerGetFileStatus(S3AFileSystem.java:3185)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.getFileStatus(S3AFileSystem.java:3053)
	at org.apache.hadoop.fs.FileSystem.exists(FileSystem.java:1760)
	at org.apach